#EJERCICIOS AZURESQL WORK ADVNETURE

1.Cual es la region con el mayor numero de  ventas en el ultimo trimestre.

In [31]:
SELECT SA.CountryRegion, CA.AddressID, SA.AddressID, SO.CustomerID, CA.CustomerID,SUM(OrderQty) AS CANTIDAD_VENTAS 
FROM SalesLT.Address SA
INNER JOIN SalesLT.CustomerAddress CA ON SA.AddressID = CA.AddressID
INNER JOIN SalesLT.SalesOrderHeader SO ON SO.CustomerID = CA.CustomerID
INNER JOIN SalesLT.SalesOrderDetail SOD ON SOD.SalesOrderID = SO.SalesOrderID
WHERE CONVERT(DATE, SO.OrderDate) BETWEEN '2008-01-01' AND '2001-01-30'
GROUP BY SA.CountryRegion,CA.AddressID, SA.AddressID, SO.CustomerID, CA.CustomerID


(0 rows affected)

Total execution time: 00:00:00.086

CountryRegion,AddressID,AddressID,CustomerID,CustomerID,CANTIDAD_VENTAS


In [40]:
SELECT SA.CountryRegion, CA.AddressID, SA.AddressID, SO.CustomerID, CA.CustomerID, SUM(SOD.OrderQty) AS CANTIDAD_VENTAS
FROM SalesLT.Address SA
INNER JOIN SalesLT.CustomerAddress CA ON SA.AddressID = CA.AddressID
INNER JOIN SalesLT.SalesOrderHeader SO ON SO.CustomerID = CA.CustomerID
INNER JOIN SalesLT.SalesOrderDetail SOD ON SOD.SalesOrderID = SO.SalesOrderID
WHERE CONVERT(DATE, SO.OrderDate) >= DATEADD(QUARTER, -1, DATEADD(QUARTER, DATEDIFF(QUARTER, 0, GETDATE()), 0))
  AND CONVERT(DATE, SO.OrderDate) < DATEADD(QUARTER, 0, DATEADD(QUARTER, DATEDIFF(QUARTER, 0, GETDATE()), 0))
GROUP BY SA.CountryRegion, CA.AddressID, SA.AddressID, SO.CustomerID, CA.CustomerID;

(0 rows affected)

Total execution time: 00:00:00.096

CountryRegion,AddressID,AddressID,CustomerID,CustomerID,CANTIDAD_VENTAS


In [42]:
WITH LastQuarter AS (
    SELECT DISTINCT
        DATEPART(YEAR, OrderDate) AS Year,
        DATEPART(QUARTER, OrderDate) AS Quarter
    FROM SalesLT.SalesOrderHeader
)
SELECT SA.CountryRegion, SUM(SOD.OrderQty) AS CANTIDAD_VENTAS
FROM SalesLT.Address SA
INNER JOIN SalesLT.CustomerAddress CA ON SA.AddressID = CA.AddressID
INNER JOIN SalesLT.SalesOrderHeader SO ON SO.CustomerID = CA.CustomerID
INNER JOIN SalesLT.SalesOrderDetail SOD ON SOD.SalesOrderID = SO.SalesOrderID
WHERE DATEPART(YEAR, SO.OrderDate) = (
    SELECT MAX(Year) FROM LastQuarter WHERE Quarter = (
        SELECT MAX(Quarter) FROM LastQuarter
    )
)
AND DATEPART(QUARTER, SO.OrderDate) = (
    SELECT MAX(Quarter) FROM LastQuarter
)
GROUP BY SA.CountryRegion;

(2 rows affected)

Total execution time: 00:00:00.106

CountryRegion,CANTIDAD_VENTAS
United Kingdom,1172
United States,915


1. Identificación del problema: Queremos filtrar los datos del último trimestre disponibles en la columna `OrderDate` de la tabla `SalesOrderHeader`, sin depender de la fecha actual.
    
2. Crear una subconsulta para obtener trimestres y años disponibles:
    
    Sql
    
    Copiar
    
    ```
    WITH LastQuarter AS (
        SELECT DISTINCT
            DATEPART(YEAR, OrderDate) AS Year,
            DATEPART(QUARTER, OrderDate) AS Quarter
        FROM SalesLT.SalesOrderHeader
    )
    
    ```
    
    Esta subconsulta (`LastQuarter`) selecciona de manera distinta todos los años y trimestres presentes en los datos de `OrderDate`. Utilizamos `DATEPART` para extraer el año y el trimestre de cada fecha.
    
3. Encontrar el año y trimestre más recientes:
    
    Sql
    
    Copiar
    
    ```
    WHERE DATEPART(YEAR, SO.OrderDate) = (
        SELECT MAX(Year) FROM LastQuarter WHERE Quarter = (
            SELECT MAX(Quarter) FROM LastQuarter
        )
    )
    AND DATEPART(QUARTER, SO.OrderDate) = (
        SELECT MAX(Quarter) FROM LastQuarter
    )
    
    ```
    
    Aquí, estamos seleccionando el mayor año (`MAX(Year)`) donde el trimestre (`Quarter`) es el mayor (`MAX(Quarter)`). Esto asegura que estamos obteniendo el último trimestre en nuestros datos.
    
4. Unir con la consulta principal: Finalmente, incorporamos la subconsulta dentro de nuestro query principal para filtrar los datos del último trimestre.
    
    Sql
    
    Copiar
    
    ```
    SELECT SA.CountryRegion, CA.AddressID, SA.AddressID, SO.CustomerID, CA.CustomerID, SUM(SOD.OrderQty) AS CANTIDAD_VENTAS
    FROM SalesLT.Address SA
    INNER JOIN SalesLT.CustomerAddress CA ON SA.AddressID = CA.AddressID
    INNER JOIN SalesLT.SalesOrderHeader SO ON SO.CustomerID = CA.CustomerID
    INNER JOIN SalesLT.SalesOrderDetail SOD ON SOD.SalesOrderID = SO.SalesOrderID
    WHERE DATEPART(YEAR, SO.OrderDate) = (
        SELECT MAX(Year) FROM LastQuarter WHERE Quarter = (
            SELECT MAX(Quarter) FROM LastQuarter
        )
    )
    AND DATEPART(QUARTER, SO.OrderDate) = (
        SELECT MAX(Quarter) FROM LastQuarter
    )
    GROUP BY SA.CountryRegion, CA.AddressID, SA.AddressID, SO.CustomerID, CA.CustomerID;
    
    ```
    

### Resumen

1. Subconsulta: Se usa para obtener todos los años y trimestres de las fechas disponibles.
    
2. Selección del último trimestre: Filtramos para obtener el año y el trimestre más recientes.
    
3. Consulta principal: Aplicamos el filtro del último trimestre en nuestro query principal.
    

Espero que este desglose te haya aclarado cómo conseguimos filtrar el último trimestre. ¿Algo más que quieras profundizar sobre este tema?

Organizar las consultas `JOIN` en SQL requiere algo de planificación y orden. Aquí tienes una guía paso a paso para organizarte como un verdadero pro:

### Paso 1: Entender los datos

- Conoce las tablas: Familiarízate con las tablas que vas a unir. Examina sus nombres de columnas y cómo se relacionan entre sí.
    
- Relaciones: Identifica las claves principales (Primary Keys) y claves foráneas (Foreign Keys).
    

### Paso 2: Definir el objetivo

- Claridad del objetivo: ¿Qué información exacta necesitas obtener después de unir las tablas? Define claramente los campos y el tipo de datos que quieres recuperar.
    

### Paso 3: Planificar las uniones

- Identificar las columnas de unión: Decide qué columnas usarás para hacer las uniones. Generalmente, estas son las claves que relacionan las tablas.
    
- Tipo de JOIN: Determina si necesitas un `INNER JOIN` (solo datos coincidentes), `LEFT JOIN` (todas las filas de la tabla izquierda y las coincidentes de la derecha), `RIGHT JOIN` (opuesto al LEFT JOIN) o `FULL JOIN` (todas las filas cuando haya una coincidencia en una de las tablas).
    

### Paso 4: Escribir el query

- Esqueleto: Empieza con un esqueleto básico del query. Incluye las cláusulas `SELECT`, `FROM` y `JOIN`.
    

Sql

Copiar

```
SELECT columna1, columna2, ...
FROM tabla1
JOIN tabla2 ON tabla1.columna_comun = tabla2.columna_comun

```

- Añadir más tablas: Añade más tablas siguiendo el mismo patrón.
    

Sql

Copiar

```
SELECT columna1, columna2, ...
FROM tabla1
JOIN tabla2 ON tabla1.columna_comun = tabla2.columna_comun
JOIN tabla3 ON tabla2.columna_comun = tabla3.columna_comun

```

### Paso 5: Filtrar y agrupar

- Filtrado: Utiliza la cláusula `WHERE` para filtrar los datos según tus necesidades.
    
- Agrupación: Si necesitas agregar datos, usa `GROUP BY` y las funciones de agregación como `SUM`, `AVG`, etc.
    

Sql

Copiar

```
SELECT columna1, columna2, SUM(columna3) AS total
FROM tabla1
JOIN tabla2 ON tabla1.columna_comun = tabla2.columna_comun
WHERE condición
GROUP BY columna1, columna2

```

### Paso 6: Ordenar

- Ordenar los resultados: Usa `ORDER BY` si necesitas ordenar los resultados.
    

Sql

Copiar

```
SELECT columna1, columna2, SUM(columna3) AS total
FROM tabla1
JOIN tabla2 ON tabla1.columna_comun = tabla2.columna_comun
WHERE condición
GROUP BY columna1, columna2
ORDER BY total DESC

```

### Paso 7: Revisar y probar

- Revisión: Revisa el query para detectar posibles errores.
    
- Prueba: Ejecuta el query en un entorno de prueba para asegurarte de que los resultados son los esperados.

2.Obtener la lista de los 10 primeros clientes:

In [43]:
SELECT TOP 10 CustomerID, FirstName, LastName, CompanyName
FROM SalesLT.Customer
ORDER BY LastName ASC;

(10 rows affected)

Total execution time: 00:00:00.707

CustomerID,FirstName,LastName,CompanyName
582,Catherine,Abel,Professional Sales and Service
29485,Catherine,Abel,Professional Sales and Service
579,Kim,Abercrombie,Riders Company
29486,Kim,Abercrombie,Riders Company
491,Frances,Adams,Area Bike Accessories
544,Jay,Adams,Valley Bicycle Specialists
29489,Frances,Adams,Area Bike Accessories
29492,Jay,Adams,Valley Bicycle Specialists
551,Samuel,Agcaoili,Vinyl and Plastic Goods Corporation
29494,Samuel,Agcaoili,Vinyl and Plastic Goods Corporation


3.Los cinco productos mas caros.

In [7]:
SELECT TOP 5 ProductNumber,ListPrice FROM SalesLT.Product
ORDER BY ListPrice DESC

(5 rows affected)

Total execution time: 00:00:00.082

ProductNumber,ListPrice
BK-R93R-62,"3578,27"
BK-R93R-44,"3578,27"
BK-R93R-48,"3578,27"
BK-R93R-52,"3578,27"
BK-R93R-56,"3578,27"


4.Contar cuantos productos hay en cada categoría.

In [10]:
SELECT PC.Name,COUNT(P.ProductID) AS CANTIDAD_PRODUCTO_TOTAL FROM SalesLT.ProductCategory PC
LEFT JOIN SalesLT.Product P ON PC.ProductCategoryID = P.ProductCategoryID
GROUP BY PC.NAME

(41 rows affected)

Total execution time: 00:00:00.091

Name,CANTIDAD_PRODUCTO_TOTAL
Bikes,0
Components,0
Clothing,0
Accessories,0
Mountain Bikes,32
Road Bikes,43
Touring Bikes,22
Handlebars,8
Bottom Brackets,3
Brakes,2


5\. Ventas totales para cada año. Resumen de ventas cada año para analoizar el crecimiento.

In [22]:
SELECT YEAR(SOH.OrderDate) AS AÑO,SUM(SOD.OrderQty) AS TOTAL_VENTAS FROM SalesLT.SalesOrderDetail SOD
LEFT JOIN SalesLT.SalesOrderHeader SOH ON SOD.SalesOrderID = SOD.SalesOrderID
GROUP BY YEAR(SOH.OrderDate)


(1 row affected)

Total execution time: 00:00:00.092

AÑO,TOTAL_VENTAS
2008,66784


6.Lista de Productos precio superior a 1000 dolares y sus Categorias